In [24]:
import os
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import json

In [27]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import json

BASE_PATH = "../data/processed"
TRAIN_PATHS = [
    "X_train_con_outliers.csv",
    "X_train_sin_outliers.csv",
    "X_train_con_outliers_norm.csv",
    "X_train_sin_outliers_norm.csv",
    "X_train_con_outliers_scal.csv",
    "X_train_sin_outliers_scal.csv"
]
TEST_PATHS = [
    "X_test_con_outliers.csv",
    "X_test_sin_outliers.csv",
    "X_test_con_outliers_norm.csv",
    "X_test_sin_outliers_norm.csv",
    "X_test_con_outliers_scal.csv",
    "X_test_sin_outliers_scal.csv"
]

# Leer los datasets de entrenamiento y prueba
TRAIN_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TRAIN_PATHS]
TEST_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TEST_PATHS]

# Leer las etiquetas (target) de entrenamiento y prueba
y_train = pd.read_csv(os.path.join(BASE_PATH, "y_train.csv")).values.ravel()
y_test = pd.read_csv(os.path.join(BASE_PATH, "y_test.csv")).values.ravel()

# Definir el número de características que deseas seleccionar
k = 5  # Puedes cambiar este valor según el número de características que desees seleccionar

# Crear una lista para almacenar los resultados
results = []

# Crear un diccionario para almacenar todas las características seleccionadas
all_selected_features = {}

for index, dataset in enumerate(TRAIN_DATASETS):
    print(f"Procesando dataset {index + 1}")

    # Selección de características usando SelectKBest con f_regression (para regresión)
    selector = SelectKBest(f_regression, k=k)
    X_train_selected = selector.fit_transform(dataset, y_train)  # Ajuste y transformación para el conjunto de entrenamiento
    X_test_selected = selector.transform(TEST_DATASETS[index])  # Transformación para el conjunto de prueba

    # Almacenar las características seleccionadas en el diccionario
    selected_features = dataset.columns[selector.get_support()].tolist()  # Obtener los nombres de las características seleccionadas
    all_selected_features[TRAIN_PATHS[index]] = selected_features  # Usar el nombre del archivo como clave

    # Crear y entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train_selected, y_train)

    # Predicciones en los datos de entrenamiento y prueba
    y_train_pred = model.predict(X_train_selected)
    y_test_pred = model.predict(X_test_selected)

    # Calcular el MSE y R² para los datos de entrenamiento
    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred)

    # Calcular el MSE y R² para los datos de prueba
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)

    # Almacenar los resultados en la lista, usando el nombre del archivo como "dataset"
    results.append(
       {
        "dataset_TRAIN": TRAIN_PATHS[index],  # Usar el nombre del archivo del dataset
        "MSE_train": mse_train,  # Guardar mse_train como número (no como string)
        "r2_train": r2_train,  # Guardar r2_train como número (no como string)
        "dataset_TEST": TEST_PATHS[index],  # Usar el nombre del archivo del dataset
        "MSE_test": mse_test,  # Guardar mse_test como número (no como string)
        "r2_test": r2_test,  # Guardar r2_test como número (no como string)
        "feat_sel": f"k{k}"
    }
    )

# Convertir la lista de resultados en un DataFrame
results_df = pd.DataFrame(results)

# Guardar los resultados 
with open(os.path.join(f"../data/results/results_k_{k}.json"), 'w') as json_file:
    json.dump(results, json_file, indent=4)

# Guardar las características seleccionadas en un solo archivo JSON
with open(os.path.join(f"../models/selected_features_k_{k}.json"), 'w') as json_file:
    json.dump(all_selected_features, json_file, indent=4)

# Imprimir el DataFrame con los resultados
results_df


Procesando dataset 1
Procesando dataset 2
Procesando dataset 3
Procesando dataset 4
Procesando dataset 5
Procesando dataset 6


,dataset_TRAIN,MSE_train,r2_train,dataset_TEST,MSE_test,r2_test,feat_sel
0,X_train_con_outliers.csv,3.707251e+07,0.729229,X_test_con_outliers.csv,3.590191e+07,0.804622,k5
1,X_train_sin_outliers.csv,3.706007e+07,0.729320,X_test_sin_outliers.csv,3.587868e+07,0.804748,k5
2,X_train_con_outliers_norm.csv,3.707251e+07,0.729229,X_test_con_outliers_norm.csv,3.590191e+07,0.804622,k5
3,X_train_sin_outliers_norm.csv,3.706007e+07,0.729320,X_test_sin_outliers_norm.csv,3.587868e+07,0.804748,k5
4,X_train_con_outliers_scal.csv,3.707251e+07,0.729229,X_test_con_outliers_scal.csv,3.590191e+07,0.804622,k5
5,X_train_sin_outliers_scal.csv,3.706007e+07,0.729320,X_test_sin_outliers_scal.csv,3.587868e+07,0.804748,k5


In [29]:
df_results = results_df
df_results = pd.DataFrame(results)

df_results["MSE_train"] = np.sqrt(df_results["MSE_train"])
df_results["MSE_test"] = np.sqrt(df_results["MSE_test"])

df_results = df_results[["MSE_train", "MSE_test", "r2_train", "r2_test"
""]]

df_results

,MSE_train,MSE_test,r2_train,r2_test
0,6088.719666,5991.820601,0.729229,0.804622
1,6087.698206,5989.881454,0.729320,0.804748
2,6088.719666,5991.820601,0.729229,0.804622
3,6087.698206,5989.881454,0.729320,0.804748
4,6088.719666,5991.820601,0.729229,0.804622
5,6087.698206,5989.881454,0.729320,0.804748
